In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from scipy import signal
from scipy.fft import fftshift

In [2]:
enter = pd.read_csv('../data/marking/enter.csv')

In [3]:
enter

,dt,inn,gtin,prid,operation_type,cnt
0,2021-11-22,F0C677A146366202906368AF9FCEC188,C238E7C33B56BA911F576777316EC9F2,F0C677A146366202906368AF9FCEC188,Импорт. Не ЕАЭС,6
1,2021-11-22,F0DC262C40FBA363CFCEF8005E4C6140,4D3423957ECDC40F70E37EB75A6A2D3E,F0DC262C40FBA363CFCEF8005E4C6140,Перемаркировка,10
2,2021-11-22,32D845A9CC2A5BFA9F77CAE0E9A905A5,F1931263F03E1D89A43A3633AAF9FB84,32D845A9CC2A5BFA9F77CAE0E9A905A5,Импорт. Не ЕАЭС,600
3,2021-11-22,CE816FDF82B121C67CAE9B79203B8BA1,0374D8CAC82E9F0D47C95FA13D68C7C6,CE816FDF82B121C67CAE9B79203B8BA1,Импорт. Не ЕАЭС,1860
4,2021-11-22,E34F3F6C9E49FE46C87D067306AAC29B,BBC31CA374A30B34CAFBFD027888A73D,E34F3F6C9E49FE46C87D067306AAC29B,РФ,1
...,...,...,...,...,...,...
740579,2022-11-21,6B8E111AB5B5C556C0AEA292ACA4D88B,476D3708FBD4E6B64F81D10BC28EA694,7D1B71037D27D5597BEA1D5077663541,Возврат,1
740580,2022-11-21,25977F1E02F92D2B5AB3605B196C7ACB,37458AD74B59A3DE08EAE571C63DD3CF,7D1B71037D27D5597BEA1D5077663541,Возврат,1
740581,2022-11-21,126F1B83C0B42598DEE7E815A6ACAC71,DB81FBEFD3EAA20E18DCCCD397717499,126F1B83C0B42598DEE7E815A6ACAC71,Возврат,2
740582,2022-11-21,0E8EEC5192C68CF67F0E02AE30AFB56A,0504A3B05E1BD8A63D3A83F13CF49658,88747153B9CE8C25EA0347DC712C1368,Возврат,1


In [14]:
px.line(enter.groupby('dt').sum())

C:\Users\simple\AppData\Local\Temp\ipykernel_26216\2199297256.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [11]:
# enter.groupby('operation_type').sum()

C:\Users\simple\AppData\Local\Temp\ipykernel_26216\1735978776.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  enter.groupby('operation_type').sum()


,cnt
operation_type,
Возврат,1378666
Импорт. ЕАЭС,3865609
Импорт. Не ЕАЭС,45547689
Маркировка остатков,1220491
Перемаркировка,1173508
Принято от физического лица,27791
РФ,68637654


In [4]:
products = pd.read_csv('../data/marking/products.csv')

In [5]:
products

,gtin,inn,product_name,product_short_name,tnved,tnved10,brand,country,volume
0,8C414C7757D587886C10DB0CCEA641A2,D0B1FE981FCC19F934C3FFD91690430F,68F60FA530914522B26E25F262EBC6D6,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,58A4B52373651DA4292AD5725D388F8A,D2D1641B196DA5477D40C0907FD5F1DA,NaN,10
1,3D8C701836C4C0A5B33D57522A7EDBAB,9B0AFDA27FCCB188768FA9B611183387,670017E2D871B8962E5456CA71F6A1D1,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,58A4B52373651DA4292AD5725D388F8A,670017E2D871B8962E5456CA71F6A1D1,NaN,НЕ КЛАССИФИЦИРОВАНО
2,B1308C75C38993B0EEA22C8FE2899C2A,86CAE1EAD8AF200ED46453FD18FA1A04,97181A85E3400C38AB223F17A5DCCFCC,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,ABB51261177B26E23AC2DE0F0508D6AB,NaN,100
3,1D71E57CA201A8ACEADBAA8D6E7039DE,86CAE1EAD8AF200ED46453FD18FA1A04,EA5CEDF8FA0F5BEBD2F047151B2DBE86,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,EDFFD44E0EC18764685E952553560EEF,NaN,100
4,5DEF4AA62907D322F85FA88839F00E8F,86CAE1EAD8AF200ED46453FD18FA1A04,75AF532015CFA92F71FF54ADEDB9BEA3,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,75AF532015CFA92F71FF54ADEDB9BEA3,NaN,50
...,...,...,...,...,...,...,...,...,...
393462,9205239C92C9E581328496DB794F60EC,8F599665A672F2CB6D29C23D4DAC7B9D,F8EE293A2AC51173AF97BE12876D6500,9199AB529CF62D4BDB7E8B1D7459001D,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,2B4A0053B03542072C3C13C9A3EC33E1,NaN,100
393463,AD775CC9CB0F39F9A5541C2CCE12E4E6,3B5BEA39AF78664AEA44752371845804,EE747123845C7B4F4EA96B3626538577,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,A9EF1048419D8E2D4848D9389275CC0E,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
393464,5EEBF058F0AADCB4095A39E8EED63CF3,3B5BEA39AF78664AEA44752371845804,1056EF3A7F6831AFF867A3C072328983,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,A9EF1048419D8E2D4848D9389275CC0E,СОЕДИНЕННОЕ КОРОЛЕВСТВО,100
393465,B54380CC5CCC0F3DDD2D448177ECB4BC,3B5BEA39AF78664AEA44752371845804,B84184EE78EB828BAE9C6C0324C0CBA6,B8AF2335E04E7BAD5168D9B39FABAC21,6D2580183CEF6C8AF1CC72E1C6E6FBC4,FAEA36DABA0E46CD081A7FA49DAD8FE0,35DE9DA6A73429012DCB8B879A24E731,ФРАНЦИЯ,75


In [6]:
retailers = pd.read_csv('../data/marking/retailers.csv')

In [7]:
retailers

,id_sp_,inn,region_code,city_with_type,city_fias_id,postal_code
0,C72AC534DBF5E6369840FFE9DA9FF403,0056F9FCFA009E493BEFEAE4FAA71D82,51,NaN,NaN,184421.0
1,591942644AE25AE4DD26CF943BB97414,81AAC456284F455FC2B141E6E8F1A76C,66,г Екатеринбург,2763c110-cb8b-416a-9dac-ad28a55b4402,620027.0
2,A6E2261F16F84FF09E2B615EDB008943,87C2FDB73C26C96266A0947C2BB714BF,62,г Рязань,86e5bae4-ef58-4031-b34f-5e9ff914cd55,390026.0
3,DC3B647A089FA064DE61E0C778F5EE45,87C2FDB73C26C96266A0947C2BB714BF,62,г Рязань,86e5bae4-ef58-4031-b34f-5e9ff914cd55,390017.0
4,D2F125BAA947B6293CD055DA1DEEA87D,87C2FDB73C26C96266A0947C2BB714BF,62,г Рыбное,91d71d8c-2f1a-43c8-bc84-d78dc4245a1b,391112.0
...,...,...,...,...,...,...
58396,7230FEFB3B05186DD7C8DB90032BA110,AA7DCE37EB49B4A0CCF74C9FE1A1563B,42,г Прокопьевск,2d8b9d01-1c4f-4987-8a0b-1eb5080c1f38,653039.0
58397,43441D66745A34C6A358084B67EA9180,380E33F3782067D65B76C4B36EDC95E1,18,NaN,NaN,427088.0
58398,F296B3072969E65CB3F836A9DAE41671,7AA85B38D45C99C4143D6D30FC69A804,42,г Прокопьевск,2d8b9d01-1c4f-4987-8a0b-1eb5080c1f38,653010.0
58399,EC2ADD23B19CB66BE2B138F175BAAA25,E139E6A1A0F51E02261249F03E0A1402,25,г Владивосток,7b6de6a5-86d0-4735-b11a-499081111af8,690090.0


In [8]:
agents = pd.read_csv('../data/marking/agents.csv')

In [9]:
agents

,inn,region_code
0,60DFB6F87ABF6232A5D97A1A827F8D88,1
1,2D13C279C7CEC509D21695F0268ECCEA,1
2,1705C73792D7A1A5ABDF1883FE851004,1
3,B4CC35BD7D8F69E665160F548EBE125D,23
4,2DB318BEA4A44280FA488A0B2AF36254,1
...,...,...
23467,FD378AE7B1D03A5B949A96BF2715B941,77
23468,CB8661DA918DFA68498D7F929C426E1E,99
23469,92ECDE80D71E459551E809AF332785AA,77
23470,86B3B09285C227598C66DE143C5AD336,78


In [10]:
movement = pd.read_csv('../data/marking/movement.csv')

In [11]:
movement

,dt,gtin,prid,sender_inn,receiver_inn,cnt_moved
0,2021-11-22,867FDFE9E03925792F329DEDBAA3955F,9246A2BE043136FC4B4A166BB0E25D3E,2E55013259E009F4FC1E5EFCF170EA8B,8D5F8E7D20AD9A8D0E8EE5769321D8C3,1
1,2021-11-22,4FB53B945150E362C307D60297066562,9246A2BE043136FC4B4A166BB0E25D3E,32B53198A919C559CB84DC0487947E80,EDE6F6D21300D96A9E207EA4F59321EF,1
2,2021-11-22,38BA9B93C5C987770AD698646D2FAB66,2DADC5DF60A9FC194A3388C1612CC967,3A87113AC25EFD88BD3407ACECFA3869,D02A2294DDF3BCF7218D6FEFDB35A906,1
3,2021-11-22,FA77213DA058666C6781164754CBC0EB,FFC432B34974A68B2845FBED9A7DEB29,C5CDC4F467B7CDA0F3E082BFFEDA9938,051C7D77FE590C5428DB57FD48C5DB9C,3
4,2021-11-22,E5635C538FE51D88504CE5CFF8BC1D2C,6E536425E534ABA7AB9FEA2ACA91A775,AE7F5C67298A67CACD2A193C9405524F,0D28C0E0C8ADAA3D9B55B557C71D8A8E,2
...,...,...,...,...,...,...
8859253,2022-11-21,F3DE4F005EF65C8E66D7AB0683B54F54,7D1B71037D27D5597BEA1D5077663541,D18B1B470E57871FAB5B4E949E256BB4,08AF18664D85C4D2D22E042F7DD3266A,3
8859254,2022-11-21,C21ED5B431F287F33AD42C71C981099E,9246A2BE043136FC4B4A166BB0E25D3E,4671D4CBD5C6CC6315746B067D64C768,D531EF78F27C40B11E0266992ADFA856,1
8859255,2022-11-21,568E5BE2AFF72BC2F47BF0C5162A52DD,3F9C20F2F0335180915DD91E8F974882,40B7AB50AEFCAF8F2B5D47D7D57185BC,E34345C4111AAC1E88642C7A25543085,1
8859256,2022-11-21,5E0E70F49C286C2E181E17E5C00C9B07,F8E0B42035183C08A9821B76843B4D6A,050C148EBA7FD16960415F2084D22EA8,9AA1C8612DC529D5DC2844F9E0F684DC,1


In [17]:
px.line(movement.groupby('dt').sum())

C:\Users\simple\AppData\Local\Temp\ipykernel_26216\3025518870.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [12]:
leave = pd.read_csv('../data/marking/leave.csv')

In [13]:
leave

,dt,gtin,prid,inn,id_sp_,type_operation,price,cnt
0,2021-11-22,5A3E5F7B2D093D1D6CB3CF93BA9AC8A6,DC7432556B58D454B4E37F0F0D1E663D,B035ED5D308BC70E5EA9342B6DBF5810,19485F700068E6A9A8593AFDE9803638,Продажа конечному потребителю в точке продаж,3500,1
1,2021-11-22,9A2202A48FC5E462C44B78B0C79E1DBD,D18B1B470E57871FAB5B4E949E256BB4,DD486FED86564603D2CD7CF9EED2C458,C2969A4C8A40C12902EAD4936E303045,Продажа конечному потребителю в точке продаж,56714,1
2,2021-11-22,877ABE9DE14E0B510DFC8DE63EEFA817,62B5EAEF59E6693F44B1074C1140BF45,62B5EAEF59E6693F44B1074C1140BF45,5637334F301773E28AE8730713E6D5F7,Продажа конечному потребителю в точке продаж,74314,1
3,2021-11-22,B004F889F17F8AB365C494B3D0D06E1A,D7326D87A37D73AF19AF53668A7116D6,D7326D87A37D73AF19AF53668A7116D6,0FF56DEC16050D3B6372F2CB1BD6DFCB,Продажа конечному потребителю в точке продаж,4999,1
4,2021-11-22,E1C0503EA9201D56D7173CB4CA4A6AFA,50955BB0B97298EEAEB1F6CF9712F37A,39FFA592BE77CA7DE37C65C1B8423E8C,9E660AA1DE428BA88D1942DB2824BDAB,Продажа конечному потребителю в точке продаж,29857,1
...,...,...,...,...,...,...,...,...
52095176,2022-04-07,5D74B67E1DAC42C7526F27A6BDA73FF6,DA62EC79660CF21AC37A260DA6F642C4,E147E089C99F3A5EFEEF57739F60EFD1,09C6904B8ACA75E3380000EB4AC0A616,Продажа конечному потребителю в точке продаж,1128,2
52095177,2022-04-07,B1C38C41D23BA4AC631FCF073744A362,D80CBF60BE36ACE344F8350B648C688D,E147E089C99F3A5EFEEF57739F60EFD1,NaN,Продажа конечному потребителю в точке продаж,3571,1
52095178,2022-04-07,2C654E2727AACB613F30DB2FF30844A1,6BC47888CB4E777EE8DF0B940FFA18C7,E147E089C99F3A5EFEEF57739F60EFD1,2FF1ACF094C477DF28C6059CA11424BF,Продажа конечному потребителю в точке продаж,4196,1
52095179,2022-04-07,859C541431985132C29F457945761242,DA62EC79660CF21AC37A260DA6F642C4,E147E089C99F3A5EFEEF57739F60EFD1,07905E35AB800714D847D6DEBAE46F47,Продажа конечному потребителю в точке продаж,985,3


In [22]:
# leave.groupby('type_operation').sum()


KeyboardInterrupt



In [ ]:
px.line(leave.groupby('dt').sum())

In [63]:
# leave.groupby('gtin').sum()

C:\Users\simple\AppData\Local\Temp\ipykernel_12440\3456055669.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,price,cnt
gtin,,
000083D6DD3B1F4374EBEF0C7C50BDE6,422006,3
00010BDFFC01E26A2B5219E78C57FA88,1271,1
00011FF207D241A71A72B220A660A283,4285,1
0001B07FC29F1AB5C4166476EBD56719,67185,1
0001D5D3A8AA4B6C59873121AA283CB7,19700,1
...,...,...
FFFE551209CE6F829781B0587B9F36A6,142,1
FFFE5EC3380D332B6DE6DDFF3523012D,10714,1
FFFE946CAF6AA694882305E2D09ADF0A,453946,26


In [ ]:
# leave[leave['type_operation'] == 'Конечная продажа организации']['price'].mean()
# # Продажа конечному потребителю в точке продаж // mean = 29494
# # Дистанционная продажа конечному потребителю // mean = 406
# # Конечная продажа организации // 450


In [46]:
# leave[leave['price'] == 0]['cnt'].sum()

26296457

In [45]:
# leave[leave['price'] != 0]['cnt'].sum()

77966459

In [35]:
# anomalies = leave[leave['price'] != 0].groupby(by='gtin').std(numeric_only=True)
# anomalies

,price,cnt
gtin,,
000083D6DD3B1F4374EBEF0C7C50BDE6,25874.963987,0.000000
00010BDFFC01E26A2B5219E78C57FA88,NaN,NaN
00011FF207D241A71A72B220A660A283,NaN,NaN
0001B07FC29F1AB5C4166476EBD56719,NaN,NaN
0001D5D3A8AA4B6C59873121AA283CB7,NaN,NaN
...,...,...
FFFE3EBB1713FF50C0A77440901BFF67,0.000000,1.414214
FFFE551209CE6F829781B0587B9F36A6,NaN,NaN
FFFE5EC3380D332B6DE6DDFF3523012D,NaN,NaN


In [31]:
# enter1 = enter
# # enter1 = enter[enter['operation_type'] == 'РФ']
# enter1.groupby('prid').count()

# dft1 = enter1[enter1['dt'] < '2021-12-01'].groupby('prid').sum()
# dft2 = enter1[enter1['dt'] >= '2021-12-01'][enter1['dt'] < '2022-01-01'].groupby('prid').sum()
# dft3 = enter1[enter1['dt'] >= '2022-01-01'][enter1['dt'] < '2022-02-01'].groupby('prid').sum()
# dft4 = enter1[enter1['dt'] >= '2022-02-01'][enter1['dt'] < '2022-03-01'].groupby('prid').sum()
# dft5 = enter1[enter1['dt'] >= '2022-03-01'][enter1['dt'] < '2022-04-01'].groupby('prid').sum()
# dft6 = enter1[enter1['dt'] >= '2022-04-01'][enter1['dt'] < '2022-05-01'].groupby('prid').sum()
# dft7 = enter1[enter1['dt'] >= '2022-05-01'][enter1['dt'] < '2022-06-01'].groupby('prid').sum()
# dft8 = enter1[enter1['dt'] >= '2022-06-01'][enter1['dt'] < '2022-07-01'].groupby('prid').sum()
# dft9 = enter1[enter1['dt'] >= '2022-07-01'][enter1['dt'] < '2022-08-01'].groupby('prid').sum()
# dft10 = enter1[enter1['dt'] >= '2022-08-01'][enter1['dt'] < '2022-09-01'].groupby('prid').sum()
# dft11 = enter1[enter1['dt'] >= '2022-09-01'][enter1['dt'] < '2022-10-01'].groupby('prid').sum()
# dft12 = enter1[enter1['dt'] >= '2022-10-01'].groupby('prid').sum()

C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2687580401.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dft1 = enter1[enter1['dt'] < '2021-12-01'].groupby('prid').sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2687580401.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft2 = enter1[enter1['dt'] >= '2021-12-01'][enter1['dt'] < '2022-01-01'].groupby('prid').sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2687580401.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dft2 = enter1[enter1['dt'] >= '2021-12-01'][enter1['dt'] < '2022-01-01'].groupby('pri

In [103]:
# dfq = dft1
# for dft in [dft2,dft3,dft4,dft5,dft6,dft7,dft8,dft9,dft10,dft11,dft12,enter1.groupby('prid').sum()]:
#     dfq = dfq.merge(dft, left_on='prid', right_on='prid', how='outer')
# dfq.fillna(0)

C:\Users\simple\AppData\Local\Temp\ipykernel_18384\128315148.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\simple\AppData\Local\Temp\ipykernel_18384\128315148.py:3: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.

C:\Users\simple\AppData\Local\Temp\ipykernel_18384\128315148.py:3: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.

C:\Users\simple\AppData\Local\Temp\ipykernel_18384\128315148.py:3: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.

C:\Users\simple\AppData\Local\Temp\

,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt
prid,,,,,,,,,,,,,
097DAED1F415A6F241A8BEA287C13AA8,1830.0,2465.0,1315.0,1535.0,469.0,1941.0,594.0,1409.0,1320.0,2773.0,976.0,4535.0,21162
1299A11C9B4DB5B9BCC4096F9C312077,130.0,389.0,134.0,273.0,802.0,175.0,489.0,209.0,283.0,71.0,278.0,481.0,3714
13FE7995BC071A827E42950EF6F309FB,6137.0,3355.0,18053.0,1709.0,3873.0,955.0,7892.0,2514.0,2648.0,2421.0,1648.0,3414.0,54619
15466B211970D7DA65CD0E0E8F41D767,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
169A32389853F886723D0DDF28A8B144,63.0,1181.0,383.0,428.0,1715.0,1230.0,376.0,1352.0,803.0,955.0,617.0,946.0,10049
...,...,...,...,...,...,...,...,...,...,...,...,...,...
D628C8F02CA907CCAA4452E8787726AD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6
DDB3806205F8A7927AD674A863643FC1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,530.0,530
EADEEBDB22BC66B0BB003503C2180359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3500.0,3500


In [37]:
# leave1 = leave.drop(['inn', 'prid', 'type_operation', 'gtin', 'price'], axis=1)
# retailers1 = retailers.drop(['postal_code'], axis=1)
#
# dft1 = leave1[leave1['dt'] < '2021-12-01'].merge(retailers1).groupby('region_code').sum()
# dft2 = leave1[leave1['dt'] >= '2021-12-01'][leave1['dt'] < '2022-01-01'].merge(retailers1).groupby('region_code').sum()
# dft3 = leave1[leave1['dt'] >= '2022-01-01'][leave1['dt'] < '2022-02-01'].merge(retailers1).groupby('region_code').sum()
# dft4 = leave1[leave1['dt'] >= '2022-02-01'][leave1['dt'] < '2022-03-01'].merge(retailers1).groupby('region_code').sum()
# dft5 = leave1[leave1['dt'] >= '2022-03-01'][leave1['dt'] < '2022-04-01'].merge(retailers1).groupby('region_code').sum()
# dft6 = leave1[leave1['dt'] >= '2022-04-01'][leave1['dt'] < '2022-05-01'].merge(retailers1).groupby('region_code').sum()
# dft7 = leave1[leave1['dt'] >= '2022-05-01'][leave1['dt'] < '2022-06-01'].merge(retailers1).groupby('region_code').sum()
# dft8 = leave1[leave1['dt'] >= '2022-06-01'][leave1['dt'] < '2022-07-01'].merge(retailers1).groupby('region_code').sum()
# dft9 = leave1[leave1['dt'] >= '2022-07-01'][leave1['dt'] < '2022-08-01'].merge(retailers1).groupby('region_code').sum()
# dft10 = leave1[leave1['dt'] >= '2022-08-01'][leave1['dt'] < '2022-09-01'].merge(retailers1).groupby('region_code').sum()
# dft11 = leave1[leave1['dt'] >= '2022-09-01'][leave1['dt'] < '2022-10-01'].merge(retailers1).groupby('region_code').sum()
# dft12 = leave1[leave1['dt'] >= '2022-10-01'].merge(retailers1).groupby('region_code').sum()
# dft_all = leave1.merge(retailers1).groupby('region_code').sum()

C:\Users\simple\AppData\Local\Temp\ipykernel_1160\1712178473.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dft1 = leave1[leave1['dt'] < '2021-12-01'].merge(retailers1).groupby('region_code').sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\1712178473.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft2 = leave1[leave1['dt'] >= '2021-12-01'][leave1['dt'] < '2022-01-01'].merge(retailers1).groupby('region_code').sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\1712178473.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dft2 = leave1[leave1['dt'] >= '2021

In [36]:
# dfq = dft1
# for dft in [dft2, dft3, dft4, dft5, dft6, dft7, dft8, dft9, dft10, dft11, dft12, dft_all]:
#     dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
# dfq.fillna(0)

C:\Users\simple\AppData\Local\Temp\ipykernel_1160\3468935475.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'price_x', 'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\3468935475.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'price_x', 'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\3468935475.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'price_x', 'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\3468935475.py:

,price_x,cnt_x,price_y,cnt_y,price_x,cnt_x,price_y,cnt_y,price_x,cnt_x,...,price_x,cnt_x,price_y,cnt_y,price_x,cnt_x,price_y,cnt_y,price,cnt
region_code,,,,,,,,,,,,,,,,,,,,,
1,98853929,3792,632933720,25649,241236200,8759,277752530,12695,364343610,12512,...,221993023,9553,232807293,9562,229086591,9257,443095319,16668,3280823463,131052
2,383373613,24643,1932317438,133080,807781200,56754,1007078123,90047,1420206337,85421,...,996319941,64294,1074439163,73122,985558795,70576,1845327239,123796,12895114669,885675
3,86816164,4338,467828635,23305,195105264,10097,235574671,17077,332845408,15631,...,260686508,12926,290550874,14101,242887181,13110,490776158,25726,3129834403,166421
4,16615198,890,110751112,5983,45118847,2446,50816291,3688,72988779,3756,...,55431069,3074,54498875,3246,47320991,2836,82764454,4961,649465039,38180
5,189467715,8953,906435021,25591,463456953,14418,564071994,20258,644347667,15234,...,487444803,10167,571283769,16442,437824006,17957,755316358,23542,6109666225,190957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,587615228,21662,3439124333,135853,1424612768,52042,1870342278,79681,2613723640,84542,...,1527643132,57326,1488025827,55363,1415710267,53337,3095272253,108247,21147003692,802296
87,5429373,108,57987068,1057,12632981,319,25590211,538,39353146,774,...,16274850,328,17845393,357,18757477,420,29892092,648,274391946,5660
89,208689321,6674,1258470715,42144,501339113,16660,716055572,25286,1155093402,30541,...,504158467,16261,494251658,15848,534545769,17234,1095724908,36297,8027019758,257660


In [47]:
# enter3 = enter.drop([], axis=1)
# agents1 = agents.drop([], axis=1)
#
# dft1 = enter3[enter3['dt'] < '2021-12-01'].merge(agents1).groupby('region_code').sum()
# dft2 = enter3[enter3['dt'] >= '2021-12-01'][enter3['dt'] < '2022-01-01'].merge(agents1).groupby('region_code').sum()
# dft3 = enter3[enter3['dt'] >= '2022-01-01'][enter3['dt'] < '2022-02-01'].merge(agents1).groupby('region_code').sum()
# dft4 = enter3[enter3['dt'] >= '2022-02-01'][enter3['dt'] < '2022-03-01'].merge(agents1).groupby('region_code').sum()
# dft5 = enter3[enter3['dt'] >= '2022-03-01'][enter3['dt'] < '2022-04-01'].merge(agents1).groupby('region_code').sum()
# dft6 = enter3[enter3['dt'] >= '2022-04-01'][enter3['dt'] < '2022-05-01'].merge(agents1).groupby('region_code').sum()
# dft7 = enter3[enter3['dt'] >= '2022-05-01'][enter3['dt'] < '2022-06-01'].merge(agents1).groupby('region_code').sum()
# dft8 = enter3[enter3['dt'] >= '2022-06-01'][enter3['dt'] < '2022-07-01'].merge(agents1).groupby('region_code').sum()
# dft9 = enter3[enter3['dt'] >= '2022-07-01'][enter3['dt'] < '2022-08-01'].merge(agents1).groupby('region_code').sum()
# dft10 = enter3[enter3['dt'] >= '2022-08-01'][enter3['dt'] < '2022-09-01'].merge(agents1).groupby('region_code').sum()
# dft11 = enter3[enter3['dt'] >= '2022-09-01'][enter3['dt'] < '2022-10-01'].merge(agents1).groupby('region_code').sum()
# dft12 = enter3[enter3['dt'] >= '2022-10-01'].merge(agents1).groupby('region_code').sum()
# dft_all = enter3.merge(agents1).groupby('region_code').sum()

C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2000614052.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dft1 = enter3[enter3['dt'] < '2021-12-01'].merge(agents1).groupby('region_code').sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2000614052.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft2 = enter3[enter3['dt'] >= '2021-12-01'][enter3['dt'] < '2022-01-01'].merge(agents1).groupby('region_code').sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2000614052.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dft2 = enter3[enter3['dt'] >= '2021-12-01

In [48]:
# dfq = dft1
# for dft in [dft2, dft3, dft4, dft5, dft6, dft7, dft8, dft9, dft10, dft11, dft12, dft_all]:
#     dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
# dfq.fillna(0)

C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2511893648.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2511893648.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2511893648.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  dfq = dfq.merge(dft, left_on='region_code', right_on='region_code', how='outer')
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\2511893648.py:3: FutureWarning: Passing 'suffix

,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt
region_code,,,,,,,,,,,,,
1,1400.0,5325.0,1315.0,6305.0,344.0,3321.0,2667.0,1806.0,3228.0,2942.0,3508.0,4819.0,36980
2,1522.0,413.0,602.0,1931.0,1060.0,1615.0,232.0,969.0,1702.0,234.0,483.0,2112.0,12875
3,71.0,269.0,117.0,291.0,72.0,68.0,94.0,71.0,71.0,43.0,50.0,71.0,1288
5,5265.0,78.0,49.0,633.0,79.0,54.0,61.0,46.0,68.0,75.0,96.0,81.0,6585
7,1.0,7.0,2.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.0,1.0,0.0,1.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,9
41,0.0,10.0,11.0,9.0,11.0,3.0,5.0,5.0,4.0,7.0,3.0,8.0,76
92,0.0,715.0,106.0,0.0,2.0,10.0,1.0,2.0,1.0,3.0,4.0,6.0,850


In [41]:
# enter2 = enter
# # enter2 = enter[enter['operation_type'] == 'РФ']
# enter2.groupby('prid').count()
#
# dft1 = enter2[enter2['dt'] < '2021-12-01'].sum()
# dft2 = enter2[enter2['dt'] >= '2021-12-01'][enter2['dt'] < '2022-01-01'].sum()
# dft3 = enter2[enter2['dt'] >= '2022-01-01'][enter2['dt'] < '2022-02-01'].sum()
# dft4 = enter2[enter2['dt'] >= '2022-02-01'][enter2['dt'] < '2022-03-01'].sum()
# dft5 = enter2[enter2['dt'] >= '2022-03-01'][enter2['dt'] < '2022-04-01'].sum()
# dft6 = enter2[enter2['dt'] >= '2022-04-01'][enter2['dt'] < '2022-05-01'].sum()
# dft7 = enter2[enter2['dt'] >= '2022-05-01'][enter2['dt'] < '2022-06-01'].sum()
# dft8 = enter2[enter2['dt'] >= '2022-06-01'][enter2['dt'] < '2022-07-01'].sum()
# dft9 = enter2[enter2['dt'] >= '2022-07-01'][enter2['dt'] < '2022-08-01'].sum()
# dft10 = enter2[enter2['dt'] >= '2022-08-01'][enter2['dt'] < '2022-09-01'].sum()
# dft11 = enter2[enter2['dt'] >= '2022-09-01'][enter2['dt'] < '2022-10-01'].sum()
# dft12 = enter2[enter2['dt'] >= '2022-10-01'].sum()
# dft_all = enter2.sum()

C:\Users\simple\AppData\Local\Temp\ipykernel_1160\4247517951.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft2 = enter2[enter2['dt'] >= '2021-12-01'][enter2['dt'] < '2022-01-01'].sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\4247517951.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft3 = enter2[enter2['dt'] >= '2022-01-01'][enter2['dt'] < '2022-02-01'].sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\4247517951.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft4 = enter2[enter2['dt'] >= '2022-02-01'][enter2['dt'] < '2022-03-01'].sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\4247517951.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft5 = enter2[enter2['dt'] >= '2022-03-01'][enter2['dt'] < '2022-04-01'].sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\4247517951.py:6: UserWarning: Boolean Series key will 

In [40]:
# dfq = dft1
# for dft in [dft2,dft3,dft4,dft5,dft6,dft7,dft8,dft9,dft10,dft11,dft12,dft_all]:
#     dfq = dfq.merge(dft, left_on='prid', right_on='prid', how='outer')
# dfq.fillna(0)

C:\Users\simple\AppData\Local\Temp\ipykernel_1160\1541370910.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  for dft in [dft2,dft3,dft4,dft5,dft6,dft7,dft8,dft9,dft10,dft11,dft12,enter1.groupby('prid').sum()]:


AttributeError: 'Series' object has no attribute 'merge'

In [63]:
# leave4 = leave[leave['type_operation'] == 'Списание / Вывод из оборота без получателя']
#
# dft1 = leave4[leave4['dt'] < '2021-12-01'].groupby(by=lambda x: 1).sum()
# dft2 = leave4[leave4['dt'] >= '2021-12-01'][leave4['dt'] < '2022-01-01'].groupby(by=lambda x: 1).sum()
# dft3 = leave4[leave4['dt'] >= '2022-01-01'][leave4['dt'] < '2022-02-01'].groupby(by=lambda x: 1).sum()
# dft4 = leave4[leave4['dt'] >= '2022-02-01'][leave4['dt'] < '2022-03-01'].groupby(by=lambda x: 1).sum()
# dft5 = leave4[leave4['dt'] >= '2022-03-01'][leave4['dt'] < '2022-04-01'].groupby(by=lambda x: 1).sum()
# dft6 = leave4[leave4['dt'] >= '2022-04-01'][leave4['dt'] < '2022-05-01'].groupby(by=lambda x: 1).sum()
# dft7 = leave4[leave4['dt'] >= '2022-05-01'][leave4['dt'] < '2022-06-01'].groupby(by=lambda x: 1).sum()
# dft8 = leave4[leave4['dt'] >= '2022-06-01'][leave4['dt'] < '2022-07-01'].groupby(by=lambda x: 1).sum()
# dft9 = leave4[leave4['dt'] >= '2022-07-01'][leave4['dt'] < '2022-08-01'].groupby(by=lambda x: 1).sum()
# dft10 = leave4[leave4['dt'] >= '2022-08-01'][leave4['dt'] < '2022-09-01'].groupby(by=lambda x: 1).sum()
# dft11 = leave4[leave4['dt'] >= '2022-09-01'][leave4['dt'] < '2022-10-01'].groupby(by=lambda x: 1).sum()
# dft12 = leave4[leave4['dt'] >= '2022-10-01'].groupby(by=lambda x: 1).sum()
# dft_all = leave4.groupby(by=lambda x: 1).sum()

C:\Users\simple\AppData\Local\Temp\ipykernel_1160\1729153477.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dft1 = leave4[leave4['dt'] < '2021-12-01'].groupby(by=lambda x: 1).sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\1729153477.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dft2 = leave4[leave4['dt'] >= '2021-12-01'][leave4['dt'] < '2022-01-01'].groupby(by=lambda x: 1).sum()
C:\Users\simple\AppData\Local\Temp\ipykernel_1160\1729153477.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dft2 = leave4[leave4['dt'] >= '2021-12-01'][leave4['dt'] < '2022-01-0

In [68]:
# dfq = dft1
# for dft in [dft2,dft3,dft4,dft5,dft6,dft7,dft8,dft9,dft10,dft11,dft12,dft_all]:
#     dfq = dfq.merge(dft, how='outer')
# dfq.fillna(0)

,price,cnt
0,201140,12737
1,160678,73801
2,1289885,38066
3,3394679,37404
4,25379957,61005
5,572682,73206
6,1012432,38926
7,666949,53131
8,717607,47111
9,2200779,117066


In [25]:
# @WIP: По месяцам
ranges = [
    ['2021-11-21', '2021-12-21'],
    ['2021-12-21', '2022-01-21'],
    ['2022-01-21', '2022-02-21'],
    ['2022-02-21', '2022-03-21'],
    ['2022-03-21', '2022-04-21'],
    ['2022-04-21', '2022-05-21'],
    ['2022-05-21', '2022-06-21'],
    ['2022-06-21', '2022-07-21'],
    ['2022-07-21', '2022-08-21'],
    ['2022-08-21', '2022-09-21'],
    ['2022-09-21', '2022-10-21'],
    ['2022-10-21', '2022-11-21'],
]

def incoming():
    px.line(enter.groupby('dt').sum()).show()
    # px.line(enter[enter['operation_type'] == 'РФ'].groupby('dt').sum().cumsum()).show()
# incoming()

def outgoing():
    px.line(leave.groupby('dt').sum()).show()
    # px.line(leave[leave['type_operation'] == 'Продажа конечному потребителю в точке продаж'].groupby('dt').sum().cumsum()).show()
# outgoing()

def production1():
    base = enter.drop(['inn', 'gtin', 'prid'], axis=1)
    accum = base.iloc[0:0]
    for [a, b] in ranges:
        step = base[base['dt'] >= a][base['dt'] < b].groupby('operation_type').sum()
        accum = accum.merge(step, left_on='operation_type', right_on='operation_type', how='outer')
    accum = accum.fillna(0)
    accum = accum.drop(['dt'], axis=1)
    px.line(accum, x='operation_type')
    return accum

production1()

C:\Users\simple\AppData\Local\Temp\ipykernel_18756\2920654476.py:31: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\simple\AppData\Local\Temp\ipykernel_18756\2920654476.py:31: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_18756\2920654476.py:31: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\simple\AppData\Local\Temp\ipykernel_18756\2920654476.py:31: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\simple\AppData\Local\Temp\ipykernel_18756\2920654476.py:31: FutureWarning:

The def

,operation_type,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt_x,cnt_y,cnt
0,Возврат,0.0,45146,55666,55790,48419,64791,101601,68197,48676,38178,47992,175326,589978
1,Импорт. ЕАЭС,0.0,274836,289637,393076,158584,209036,143757,227012,188311,338460,272581,864431,486446
2,Импорт. Не ЕАЭС,0.0,7271735,5593901,5783887,3725962,2278708,1419000,1825803,2137704,3636589,3343695,4153232,4075945
3,Маркировка остатков,0.0,735056,51625,50893,132786,2693,2327,2209,1543,1458,3708,235270,897
4,Перемаркировка,0.0,110010,70625,61164,167073,137086,151428,105651,53845,70858,66223,79948,92697
5,Принято от физического лица,0.0,1,72,257,219,366,10138,2587,2059,1901,4079,3192,2663
6,РФ,0.0,7212253,5790051,7908114,6539774,6049034,4637601,4412451,3746909,4840377,5037104,6097860,5916227
